# lv60k Fine-tuning

음성을 fft로 변환하여 학습에 사용해본다.

In [1]:
import torch
import torchaudio
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn # 인공 신경망 모델들 모아놓은 모듈
import torch.nn.functional as F #그중 자주 쓰이는것들을 F로
from torchvision import transforms, datasets
import cv2
from torchvision import transforms, datasets
import pandas as pd
import os
from glob import glob
import torchvision.models as models
import sys
import math



p = os.path.abspath('../') # 상위 폴더를 사용하기 위해서.
sys.path.insert(1, p)
from pytorchtools.pytorchtools import EarlyStopping # 상위 폴더에 추가된 모듈.

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
#DEVICE = torch.device('cpu')
print('Using Pytorch version : ',torch.__version__,' Device : ',DEVICE)

Using Pytorch version :  1.10.2  Device :  cuda


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import numpy as np
import librosa, librosa.display 
import matplotlib.pyplot as plt
#window sizde : FFT를 할때 참조할 그래프 길이 ( 프레임 하나당 sample 수 )
#자연어 처리에서는 25ms 사용. https://ahnjg.tistory.com/93
#초당 50000hz 중 1250개씩 윈도우 사이즈로 사용.
sr=50000
win_length =  np.int64(50000/40) # 1250
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.

# data 분류

In [4]:
#1. train, test 나누기

import os
from glob import glob
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.model_selection import train_test_split # train , test 분리에 사용.


pathology = glob('../../voice_data/fusion/pathology/phrase/*.wav')
healthy = glob('../../voice_data/fusion/healthy/phrase/*.wav')
print("Pathology : ",len(pathology))
print("Healthy: ",len(healthy))


pathology= [ path.split("\\")[-1] for path in pathology] # path 데이터 변환.
healthy= [ path.split("\\")[-1] for path in healthy] # path 데이터 변환.
 # path 데이터 변환


X = pathology+healthy # path 데이터 합
print("총 데이터수 : ",len(X))
Y = [] # 라벨
for idx,x in enumerate(X):
    if idx<1193:
        Y.append("pathology")
    else:
        Y.append("healthy")

X, X_test, Y, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, stratify=Y, random_state=456)
#stratify를 넣어서, test에도 라벨별 잘 분류되게 한다.

print("---")
print("훈련 셋 : ",len(Y),Counter(Y))
print("테스트 셋 : ",len(Y_test),Counter(Y_test))
print("---")

Pathology :  1193
Healthy:  634
총 데이터수 :  1827
---
훈련 셋 :  1461 Counter({'pathology': 954, 'healthy': 507})
테스트 셋 :  366 Counter({'pathology': 239, 'healthy': 127})
---


In [5]:
#1. train, test 나누기
#stratified kfold
import os
import random #데이터 shuffle 사용
from glob import glob
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from collections import Counter



skf_iris = StratifiedKFold(n_splits=5,shuffle=True,random_state=456)
cnt_iter = 0

X_train_list = [] #데이터 셋 보관
Y_train_list = []

X_valid_list = []
Y_valid_list = []

for train_idx, test_idx in skf_iris.split(X,Y):
    
    #split으로 반환된 인덱스를 이용하여, 학습 검증용 테스트 데이터 추출
    cnt_iter += 1
    X_train, X_valid = [X[idx] for idx in train_idx.tolist() ], [X[idx] for idx in test_idx.tolist() ]
    Y_train, Y_valid = [Y[idx] for idx in train_idx.tolist() ], [Y[idx] for idx in test_idx.tolist() ]
    
    X_train_list.append(X_train)
    X_valid_list.append(X_valid)
    
    Y_train_list.append(Y_train)
    Y_valid_list.append(Y_valid)
    
    
    #학습 및 예측
    
    label_train = Y_train
    label_test = Y_valid
    unique_train, train_counts = np.unique(label_train, return_counts = True)
    unique_test, test_counts = np.unique(label_test, return_counts = True)
    
    uniq_cnt_train = dict(zip(unique_train, train_counts))
    uniq_cnt_test = dict(zip(unique_test, test_counts))
    
    
    
    print('교차 검증 : {}'.format(cnt_iter))
    print('학습 레이블 데이터 분포 : \n', uniq_cnt_train)
    print('검증 레이블 데이터 분포 : \n', uniq_cnt_test,'\n')



교차 검증 : 1
학습 레이블 데이터 분포 : 
 {'healthy': 405, 'pathology': 763}
검증 레이블 데이터 분포 : 
 {'healthy': 102, 'pathology': 191} 

교차 검증 : 2
학습 레이블 데이터 분포 : 
 {'healthy': 406, 'pathology': 763}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 191} 

교차 검증 : 3
학습 레이블 데이터 분포 : 
 {'healthy': 406, 'pathology': 763}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 191} 

교차 검증 : 4
학습 레이블 데이터 분포 : 
 {'healthy': 406, 'pathology': 763}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 191} 

교차 검증 : 5
학습 레이블 데이터 분포 : 
 {'healthy': 405, 'pathology': 764}
검증 레이블 데이터 분포 : 
 {'healthy': 102, 'pathology': 190} 



In [6]:
#2. random over sampling
for i in range(5):
    X_temp = np.array(X_train_list[i]).reshape(-1,1)#각 데이터를 다 행으로 넣음. (1194,1)
    #Y = np.array(Y)
    ros = RandomOverSampler(random_state = 123)
    X_res,Y_res = ros.fit_resample(X_temp,Y_train_list[i])
    
    print("\n fold{} ".format(i))
    print('before dataset shape {}'.format(Counter(Y_train_list[i])) )
    print('Resampled dataset shape {}'.format(Counter(Y_res)) )
    
    #원래대로 돌리기
    X_res=X_res.reshape(1, -1)
    X_train_list[i]=X_res[0].tolist()
    Y_train_list[i]=Y_res




 fold0 
before dataset shape Counter({'pathology': 763, 'healthy': 405})
Resampled dataset shape Counter({'pathology': 763, 'healthy': 763})

 fold1 
before dataset shape Counter({'pathology': 763, 'healthy': 406})
Resampled dataset shape Counter({'pathology': 763, 'healthy': 763})

 fold2 
before dataset shape Counter({'pathology': 763, 'healthy': 406})
Resampled dataset shape Counter({'pathology': 763, 'healthy': 763})

 fold3 
before dataset shape Counter({'pathology': 763, 'healthy': 406})
Resampled dataset shape Counter({'pathology': 763, 'healthy': 763})

 fold4 
before dataset shape Counter({'pathology': 764, 'healthy': 405})
Resampled dataset shape Counter({'pathology': 764, 'healthy': 764})


In [7]:
import pickle

 
#load
with open("../../voice_data/fusion/phrase_sig_dict.pickle","rb") as fr:
    phrase_dict = pickle.load(fr)

    

# data set

In [8]:
from torch.utils.data import Dataset, DataLoader

classes = ["pathology","healthy"]
sr=16000
win_length =  np.int64(sr/40) # 1250
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.



class svd_dataset(Dataset):
    def __init__(self,data_path_list,classes,data_num,training):
        #클래스에서 사용할 인자를 받아 인스턴스 변수로 저장하는 일을 한다.
        #예를들면, 이미지의 경로 리스트를 저장하는 일을 하게 된다.
        
        #data_num : k 개 데이터 셋 중 어떤것을 쓸지
        #test인지 아닌지.
        
        self.path_list = data_path_list[data_num]
        self.data_num = data_num
        self.training = training
        self.label = svd_dataset.get_label(self.path_list,training,data_num)
        self.classes=classes
        
    
    @classmethod
    def get_label(cls,data_path_list,training,data_num):
        label_list=[]
        
        if training:
            for idx,x in enumerate(data_path_list):
                label_list.append(Y_train_list[data_num][idx])
        else:
            for idx,x in enumerate(data_path_list):
                label_list.append(Y_valid_list[data_num][idx])
        #print(label_list)
        return label_list
    
    def __len__(self):
        return len(self.path_list)
        #데이터 셋의 길이를 정수로 반환한다.     
        
    def __getitem__(self, idx):
        """
        1. path를 받아서, signal 변환
        2. raw 데이터 바로 출력.
        
        """

        sig = phrase_dict[self.path_list[idx]]
        sig_length = sig.size #sig length 리턴에 사용. feature 추출및 학습에 사용.
        sig=np.fft.fft(sig)
        magnitude = np.abs(sig)
        pad1d=lambda a, i: a[0:i] if a.shape[0] > i else np.hstack((a, np.zeros((i-a.shape[0]))))
        left_spectrum=pad1d(magnitude,2*sr)
        
        left_spectrum = torch.from_numpy(left_spectrum).type(torch.float32)
        
        return left_spectrum, self.classes.index(self.label[idx]),sig_length

In [9]:
# test set 제작을 위한 class
class svd_test_set(Dataset):
    def __init__(self,data_path_list,classes):
        #클래스에서 사용할 인자를 받아 인스턴스 변수로 저장하는 일을 한다.
        #예를들면, 이미지의 경로 리스트를 저장하는 일을 하게 된다.
        
        #data_num : k 개 데이터 셋 중 어떤것을 쓸지
        #test인지 아닌지.
        
        self.path_list = data_path_list
        self.label = svd_test_set.get_label(self.path_list)
        self.classes=classes

        
    
    @classmethod
    def get_label(cls,data_path_list):
        label_list=[]
        
        for idx,x in enumerate(data_path_list):
            label_list.append(Y_test[idx])
        #print(label_list)
        return label_list
    
    def __len__(self):
        return len(self.path_list)
        #데이터 셋의 길이를 정수로 반환한다. 
        
    def __getitem__(self, idx):
        """
        1. path를 받아서, signal 변환
        2. raw 데이터 바로 출력.
        
        """

        sig = phrase_dict[self.path_list[idx]]
        sig_length = sig.size #sig length 리턴에 사용. feature 추출및 학습에 사용.
        sig=np.fft.fft(sig)
        magnitude = np.abs(sig)
        pad1d=lambda a, i: a[0:i] if a.shape[0] > i else np.hstack((a, np.zeros((i-a.shape[0]))))
        left_spectrum=pad1d(magnitude,2*sr)
        left_spectrum = torch.from_numpy(left_spectrum).type(torch.float32)
        
        return left_spectrum, self.classes.index(self.label[idx]),sig_length

# data loader

In [10]:
#3. 하이퍼 파라미터
BATCH_SIZE =  16 #한 배치당 32개 음성데이터
EPOCHS = 50 # 전체 데이터 셋을 50번 반복

In [11]:
#DATA LOADER 함수가 BATCH_size 단위로 분리해 지정.

#확인을 위해 데이터셋 하나만 확인



train_loader = torch.utils.data.DataLoader(dataset = 
                                           svd_dataset(
                                               X_train_list,
                                                   classes,
                                               data_num=0,
                                               training=True
                                           ),
                                           batch_size = BATCH_SIZE,
                                           shuffle = True,
                                           ) # 순서가 암기되는것을 막기위해.

validation_loader = torch.utils.data.DataLoader(dataset = 
                                           svd_dataset(
                                               X_valid_list,
                                               classes,
                                               data_num=0,
                                               training=False
                                           ),
                                           batch_size = BATCH_SIZE,
                                           shuffle = True,) 



In [12]:
# 테스트 데이터 로더.

test_loader = torch.utils.data.DataLoader(dataset = 
                                               svd_test_set(
                                                   X_test,
                                                   classes,
                                               ),
                                               batch_size = BATCH_SIZE,
                                               shuffle = True,) 

# model

https://pytorch.org/audio/main/tutorials/speech_recognition_pipeline_tutorial.html



In [14]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

        
dat1=torch.randn((8,96000))

dat1=np.fft.fft(dat1.detach().numpy())
magnitude = np.abs(dat1)
left_spectrum = magnitude[:,:sr]
left_spectrum = torch.from_numpy(left_spectrum).type(torch.float32)

seq_len=torch.from_numpy(np.array([30000,32000,34000,36000,38000,40000,42000,44000]))
model1=torchaudio.models.wav2vec2_large_lv60k(aux_num_out=32)
model1.aux = Identity().to(DEVICE)


sample1,_=model1(left_spectrum)
sample1


tensor([[[ 8.5225e-01, -4.0867e-01,  7.4538e-01,  ..., -6.8556e-02,
          -3.7129e-01,  1.8070e+00],
         [ 7.9797e-01, -3.7286e-01,  4.9027e-01,  ..., -5.0130e-01,
          -8.2070e-01,  8.9578e-01],
         [ 7.6002e-01,  1.6560e-01,  2.4050e-01,  ...,  3.8992e-01,
           1.8064e-03,  1.6377e+00],
         ...,
         [ 1.1860e+00, -5.4033e-01,  7.4450e-01,  ...,  6.6462e-01,
          -5.8482e-01,  1.4552e+00],
         [ 8.7752e-01,  7.3099e-02,  6.8841e-01,  ...,  1.4719e-01,
          -6.5323e-01,  1.5304e+00],
         [ 7.8836e-01, -6.3953e-01,  5.8233e-03,  ..., -3.8607e-02,
          -2.3137e-01,  1.5934e+00]],

        [[ 7.4549e-01,  6.6521e-03,  5.6428e-01,  ...,  5.7456e-01,
          -8.7255e-01,  1.6658e+00],
         [ 9.3712e-01, -1.9490e-01,  9.2537e-01,  ...,  9.9763e-01,
          -7.6638e-01,  1.3044e+00],
         [ 9.3769e-01,  5.1639e-02,  4.4662e-01,  ...,  8.4617e-01,
          -7.7418e-01,  1.7852e+00],
         ...,
         [ 8.0994e-01, -3

In [40]:
dat1=torch.randn((96000))

dat1=np.fft.fft(dat1.detach().numpy())
magnitude = np.abs(dat1)
left_spectrum = magnitude[:sr]
left_spectrum = torch.from_numpy(left_spectrum).type(torch.float32)
left_spectrum.size()

torch.Size([16000])

In [48]:
int(magnitude.shape[1]/2)

48000

In [44]:
magnitude[:,:int(len(magnitude)/2)].shape

(8, 4)

In [15]:
sample1.size()

torch.Size([8, 49, 1024])

In [25]:
def nan_padding_list(hidden_feature,seq_len):
    nan_padding_array=np.ones((len(seq_len),299, hidden_feature.size()[-1] ))
    for i in range(len(seq_len)):
        nan_padding_array[i,seq_len[i]:,:]=0
    #print("seq len : ",seq_len)
    #print("pad : ",nan_padding_array)
    hidden_feature=hidden_feature*torch.from_numpy(nan_padding_array).float() # 318 * 2\n"
    #print(hidden_feature)
    return hidden_feature
    

In [28]:

sample2=nan_padding_list(sample1,seq_size)


In [34]:
sample2[sample2.nonzero(as_tuple=True)]


tensor([ 1.0372,  0.5164, -0.3926,  ..., -0.8446,  0.5000,  0.9283],
       grad_fn=<IndexBackward0>)

In [30]:
sample2[sample2.nonzero(as_tuple=True)].view(-1, sample2.shape[1],sample2.shape[2])

RuntimeError: shape '[-1, 299, 1024]' is invalid for input of size 942080

In [15]:
seq_size.type(torch.long)

tensor([93, 99, 93, 93, 93, 93, 93, 93])

In [ ]:
dat1=torch.randn((32,1024,99))
decoder = nn.Sequential(
            nn.Conv1d(1024, 1024, 150, dilation=2), #dilation2 
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Conv1d(1024, 1024, 1), #point wise
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Conv1d(1024,1024,1)
        )
sample1=decoder(dat1)
sample1.size()

In [ ]:
input_size1 = 1 # node가 299 time step ,1 차원데이터
hidden_size1 = 2 # 마지막 출력은 2 class ->bidirectional은 2배
num_layers=1 # 1 레이어 lstm


dat1=torch.randn((8,299,1024))
decoder = nn.Sequential(
            nn.Conv1d(299, 299, 512, dilation=2), #dilation2 
            nn.BatchNorm1d(299),
            nn.ReLU(),
            nn.Conv1d(299,299, 2), #point wise
            nn.BatchNorm1d(299),
            nn.ReLU(),
        )
lstm1 = nn.LSTM(input_size=input_size1,
                             hidden_size=hidden_size1,
                             num_layers=num_layers,
                             batch_first = True,
                             bidirectional=False,
                             bias=True)
sample1=decoder(dat1)
sample2,_=lstm1(sample1)
sample1.size()
sample2.size()

In [ ]:
idx=torch.tensor( [range(8),seq_size.type(torch.long)] )

In [ ]:
x[x.nonzero(as_tuple=True)].view(-1, x.shape[1])

In [16]:
sample2=np.ones((99,1024))
sample2[98,1023] = np.nan
sample2[98,1022] = np.nan
sample1[0]*torch.from_numpy(sample2)


tensor([[-1.0251e-01, -4.9478e-01,  1.1263e+00,  ..., -1.4020e+00,
         -1.3896e-01,  4.7388e-01],
        [ 2.4365e-01, -2.6183e-01,  1.8771e-01,  ..., -2.2010e+00,
         -6.5691e-01,  9.5198e-02],
        [ 5.0369e-01, -4.2158e-01,  3.6093e-01,  ..., -1.1279e+00,
         -2.4562e-01,  5.4028e-01],
        ...,
        [ 2.4150e-01, -7.7891e-01,  5.2251e-01,  ..., -9.9710e-01,
         -4.8511e-01,  9.9657e-01],
        [ 2.6540e-01, -6.2383e-01, -3.4972e-01,  ..., -6.8074e-01,
         -1.2437e-01,  6.9385e-01],
        [-1.0431e-03, -8.2086e-01, -3.2673e-02,  ..., -1.1389e+00,
                 nan,         nan]], dtype=torch.float64,
       grad_fn=<MulBackward0>)

In [ ]:
idx[1, :]

In [ ]:
sample2[4,268,:]

In [ ]:
idx[1, :]

In [ ]:

sample2[idx[0,:], idx[1, :],:]


In [ ]:
list(range(8))

In [ ]:
seq_size.type(torch.long)[range(seq_size.shape[0])]

In [ ]:
sample2[torch.tensor([range(8),]),seq_size.type(torch.long),:]

In [ ]:
seq_size.type(torch.long).unsqueeze(1).unsqueeze(0)


In [13]:
#https://pytorch.org/audio/0.9.0/models.html#baevski2020wav2vec
from torchaudio.models.wav2vec2.utils import import_huggingface_model
from transformers import Wav2Vec2ForCTC





In [14]:
# wav2word 가져오기
#https://github.com/qute012/Wav2Keyword/blob/main/downstream_kws_benchmark.py
#7. Optimizer, Objective Function
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

input_size1 = 1 # node가 299 time step ,1 차원데이터
hidden_size1 = 2 # 마지막 출력은 2 class ->bidirectional은 2배
num_layers=1 # 1 레이어 lstm


class classifier_model(nn.Module):
    def __init__(self,):
        super(classifier_model,self).__init__()
        
        self.model_wav2vec2 = torchaudio.models.wav2vec2_large_lv60k(aux_num_out=32)
        self.model_wav2vec2.load_state_dict(torch.load("../checkpoint/wav2vec2-large-lv60.pt"))
        self.model_wav2vec2.aux = Identity().to(DEVICE)
        
        self.padded_frame=299
        
        self.out_channels = 112
        self.decoder = nn.Sequential(
            nn.Conv1d(self.padded_frame, self.out_channels, 512, dilation=2), #dilation2 
            nn.BatchNorm1d(self.out_channels),
            nn.ReLU(),
            nn.Conv1d(self.out_channels, self.out_channels, 2), #point wise
            nn.BatchNorm1d(self.out_channels),
            nn.ReLU(),
        )
        
        self.lstm1 = nn.LSTM(input_size=input_size1,
                             hidden_size=hidden_size1,
                             num_layers=num_layers,
                             batch_first = True,
                             bidirectional=False,
                             bias=True)
        
        
    def forward(self,x,sig_size):
        x,last_frame = self.model_wav2vec2(x,sig_size)
        #zero_padding_array=np.ones((len(seq_len),self.padded_frame,2))
        #for i in range(len(seq_len)):
        #    zero_padding_array[i:,seq_len[i]:,:]=0
        #x=x*torch.from_numpy(zero_padding_array).float().to(DEVICE) # 318 * 2
        #b,t,c = x.shape
        #x = x.reshape(b,c,t)
        x = self.decoder(x)
        x,_ = self.lstm1(x)
        #last_frame = torch.floor(last_frame*111/299).type(torch.long)
        #print(last_frame)
        #sig_idx=torch.tensor( [range(len(sig_size)),last_frame])
        x = x[:,-1,:]
        #x = self.fc(x)
        
        return x 

def model_initialize():
    model = classifier_model()
    #원핫 인코딩값의 loss는 crossEntropyLoss로 비교
    #print(model)
    return model.to(DEVICE)

# test

다음 계획은 non zero 배열을 활용해서, mean을 계산해본다.?

In [16]:
# pad change
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x



class classifier_model(nn.Module):
    def __init__(self,):
        super(classifier_model,self).__init__()
        
        self.model_wav2vec2 = torchaudio.models.wav2vec2_large_lv60k(aux_num_out=32)
        self.model_wav2vec2.load_state_dict(torch.load("../checkpoint/wav2vec2-large-lv60.pt"))
        self.model_wav2vec2.aux = Identity().to(DEVICE)
        
        self.padded_frame=299
        
        self.fc = nn.Sequential(nn.Linear(1024,512),
                                nn.BatchNorm1d(512),
                                nn.ReLU(),
                                nn.Linear(512,50),
                                nn.BatchNorm1d(50),
                                nn.ReLU(),
                                nn.Linear(50,2))
        
    def nan_padding_list(self,hidden_feature,seq_len):
        nan_padding_array=np.ones((len(seq_len),self.padded_frame, hidden_feature.size()[-1] ))
        for i in range(len(seq_len)):
            nan_padding_array[i,seq_len[i]:,:]=0
        #print("seq len : ",seq_len)
        #print("pad : ",nan_padding_array)
        hidden_feature=hidden_feature*torch.from_numpy(nan_padding_array).float().to(DEVICE) # 318 * 2\n"
        #print(hidden_feature)
        return hidden_feature
    
    def forward(self,x,sig_size):
        x,seq_len = self.model_wav2vec2(x,sig_size)
        #print("x : ",x)
        #x = self.nan_padding_list(x,seq_len)# batch,frame,feature
        
        #x = x.sum(axis=1)/(x!=0).sum(axis=1)
        x = x.mean(axis=1)
        #print("mean : ",x)
        
        x = x.view(-1,1024)
        x=self.fc(x)
        
        return x 


def model_initialize():
    model = classifier_model()
    #원핫 인코딩값의 loss는 crossEntropyLoss로 비교
    #print(model)
    return model.to(DEVICE)

In [14]:
#baseline
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x



class classifier_model(nn.Module):
    def __init__(self,):
        super(classifier_model,self).__init__()
        
        self.model_wav2vec2 = torchaudio.models.wav2vec2_large_lv60k(aux_num_out=32)
        self.model_wav2vec2.load_state_dict(torch.load("../checkpoint/wav2vec2-large-lv60.pt"))
        self.model_wav2vec2.aux = Identity().to(DEVICE)
        
        self.padded_frame=99
        
        self.fc = nn.Sequential(nn.Linear(1024,512),
                                nn.BatchNorm1d(512),
                                nn.ReLU(),
                                nn.Linear(512,50),
                                nn.BatchNorm1d(50),
                                nn.ReLU(),
                                nn.Linear(50,2))
        
        
    def forward(self,x,sig_size):
        x,_ = self.model_wav2vec2(x)
        x = x.mean(axis=1)
        
        x = x.view(-1,1024)
        x=self.fc(x)
        
        return x 


def model_initialize():
    model = classifier_model()
    #원핫 인코딩값의 loss는 crossEntropyLoss로 비교
    #print(model)
    return model.to(DEVICE)

In [15]:
#8. 학습
def train(model,train_loader,optimizer, log_interval):
    model.train()
    correct = 0
    train_loss = 0
    for batch_idx,(image,label,sig_size) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        sig_size = sig_size.to(DEVICE)
        #데이터들 장비에 할당
        optimizer.zero_grad() # device 에 저장된 gradient 제거
        output = model(image,sig_size) # model로 output을 계산 #WAV2VEC2 학습시 SIG_SIZE 필요,
        loss = criterion(output, label) #loss 계산
        train_loss += loss.item()
        prediction = output.max(1,keepdim=True)[1] # 가장 확률이 높은 class 1개를 가져온다.그리고 인덱스만
        correct += prediction.eq(label.view_as(prediction)).sum().item()# 아웃풋이 배치 사이즈 32개라서.
        loss.backward() # loss 값을 이용해 gradient를 계산
        optimizer.step() # Gradient 값을 이용해 파라미터 업데이트.
        del output
        del image
        torch.cuda.empty_cache()

    train_loss/=len(train_loader.dataset)
    train_accuracy = 100. * correct / len(train_loader.dataset)
    return train_loss,train_accuracy


In [16]:
#9. 학습 진행하며, validation 데이터로 모델 성능확인
def evaluate(model,valid_loader):
    model.eval()
    valid_loss = 0
    correct = 0
    #no_grad : 그래디언트 값 계산 막기.
    with torch.no_grad():
        for image, label,sig_size in valid_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            sig_size = sig_size.to(DEVICE)
            
            output = model(image,sig_size) #WAV2VEC2 학습시 SIG_SIZE 필요,
            valid_loss += criterion(output, label).item()
            prediction = output.max(1,keepdim=True)[1] # 가장 확률이 높은 class 1개를 가져온다.그리고 인덱스만
            correct += prediction.eq(label.view_as(prediction)).sum().item()# 아웃풋이 배치 사이즈 32개라서.
            #true.false값을 sum해줌. item
            del output
            del image
            torch.cuda.empty_cache()
            
        valid_loss /= len(valid_loader.dataset)
        valid_accuracy = 100. * correct / len(valid_loader.dataset)
        return valid_loss,valid_accuracy


In [17]:
#데이터 로더 제작 함수

def load_data(data_ind):

    train_loader = torch.utils.data.DataLoader(dataset = 
                                               svd_dataset(
                                                   X_train_list,
                                                   classes,
                                                   data_num=data_ind,
                                                   training=True
                                               ),
                                               batch_size = BATCH_SIZE,
                                               shuffle = True,
                                               ) # 순서가 암기되는것을 막기위해.

    validation_loader = torch.utils.data.DataLoader(dataset = 
                                               svd_dataset(
                                                   X_valid_list,
                                                   classes,
                                                   data_num=data_ind,
                                                   training=False
                                               ),
                                               batch_size = BATCH_SIZE,
                                               shuffle = True,) 
    return train_loader,validation_loader



In [17]:
#10. 학습 및 평가.
# kfold 적용

train_accs = []
valid_accs = []

for data_ind in range(1,6):

    check_path = '../checkpoint/checkpoint_w2v_large_finetune_nan_'+str(data_ind)+'.pt'
    print(check_path)
    early_stopping = EarlyStopping(patience = 5, verbose = True, path=check_path)
    train_loader,validation_loader = load_data(data_ind-1)
        
    best_train_acc=0 # accuracy 기록용
    best_valid_acc=0
    
    model=model_initialize()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
    
    
    print("[{} 교차검증] 학습 시작\n ----- ".format(data_ind))
    for Epoch in range(1,EPOCHS+1):
        train_loss,train_accuracy=train(model,train_loader,optimizer,log_interval=31)
        valid_loss,valid_accuracy = evaluate(model, validation_loader)


        print("\n[EPOCH:{}]\t Train Loss:{:.4f}\t Train Acc:{:.2f} %  | \tValid Loss:{:.4f} \tValid Acc: {:.2f} %\n".
              format(Epoch,train_loss,train_accuracy,valid_loss,valid_accuracy))
        

        early_stopping(valid_loss, model)
        if -early_stopping.best_score == valid_loss:
            best_train_acc, best_valid_acc = train_accuracy,valid_accuracy
        
        if early_stopping.early_stop:
                train_accs.append(best_train_acc)
                valid_accs.append(best_valid_acc)
                print("[{} 교차검증] Early stopping".format(data_ind))
                break

        if Epoch==EPOCHS:
            #만약 early stop 없이 40 epoch라서 중지 된 경우.
            train_accs.append(best_train_acc)
            valid_accs.append(best_valid_acc)

../checkpoint/checkpoint_w2v_large_finetune_nan_1.pt
[1 교차검증] 학습 시작
 ----- 

[EPOCH:1]	 Train Loss:0.0383	 Train Acc:68.15 %  | 	Valid Loss:0.0416 	Valid Acc: 61.09 %

Validation loss decreased (inf --> 0.041553).  Saving model ...

[EPOCH:2]	 Train Loss:0.0340	 Train Acc:72.94 %  | 	Valid Loss:0.0362 	Valid Acc: 74.40 %

Validation loss decreased (0.041553 --> 0.036160).  Saving model ...

[EPOCH:3]	 Train Loss:0.0303	 Train Acc:77.13 %  | 	Valid Loss:0.0380 	Valid Acc: 71.67 %

EarlyStopping counter: 1 out of 5

[EPOCH:4]	 Train Loss:0.0268	 Train Acc:81.85 %  | 	Valid Loss:0.0726 	Valid Acc: 45.39 %

EarlyStopping counter: 2 out of 5

[EPOCH:5]	 Train Loss:0.0201	 Train Acc:85.58 %  | 	Valid Loss:0.0344 	Valid Acc: 74.40 %

Validation loss decreased (0.036160 --> 0.034401).  Saving model ...

[EPOCH:6]	 Train Loss:0.0152	 Train Acc:90.69 %  | 	Valid Loss:0.1318 	Valid Acc: 65.19 %

EarlyStopping counter: 1 out of 5

[EPOCH:7]	 Train Loss:0.0107	 Train Acc:93.38 %  | 	Valid Loss:0.10

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\USER\anaconda3\envs\local_torch\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\USER\AppData\Local\Temp/ipykernel_10548/931476925.py", line 24, in <module>
    train_loss,train_accuracy=train(model,train_loader,optimizer,log_interval=31)
  File "C:\Users\USER\AppData\Local\Temp/ipykernel_10548/2456406595.py", line 12, in train
    output = model(image,sig_size) # model로 output을 계산 #WAV2VEC2 학습시 SIG_SIZE 필요,
  File "C:\Users\USER\anaconda3\envs\local_torch\lib\site-packages\torch\nn\modules\module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "C:\Users\USER\AppData\Local\Temp/ipykernel_10548/2731383049.py", line 31, in forward
    x,_ = self.model_wav2vec2(x)
  File "C:\Users\USER\anaconda3\envs\local_torch\lib\site-packages\torch\nn\modules\module.py", line 1102, in _call_impl
    return forward_call(*inp

TypeError: object of type 'NoneType' has no len()

# 결과 확인

In [20]:
sum_valid=0
for data_ind in range(5):
    print("[{} 교차검증] train ACC : {:.4f} |\t valid ACC: {:.4f} ".format(data_ind+1,train_accs[data_ind],valid_accs[data_ind] ))
    sum_valid+=valid_accs[data_ind]
    
print("평균 검증 정확도",sum_valid/5,"%")

[1 교차검증] train ACC : 88.4010 |	 valid ACC: 82.2526 
[2 교차검증] train ACC : 77.5885 |	 valid ACC: 75.6849 
[3 교차검증] train ACC : 90.9567 |	 valid ACC: 82.5342 
[4 교차검증] train ACC : 79.7510 |	 valid ACC: 79.7945 
[5 교차검증] train ACC : 83.7696 |	 valid ACC: 75.6849 
평균 검증 정확도 79.19023797278975 %


In [ ]:
#9. 학습 진행하며, validation 데이터로 모델 성능확인
def test_evaluate(model,test_loader):
    model.eval()
    predictions = []
    answers = []
    #no_grad : 그래디언트 값 계산 막기.
    with torch.no_grad():
        for image, label,sig_len in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            sig_len = sig_len.to(DEVICE)
            output = model(image,sig_len)
            output = F.softmax(output, dim=1).data.squeeze() # softmax 적용 (모델을 통과는 했지만, criterion는 안통과함.)
            prediction = output.max(1,keepdim=True)[1] # 가장 확률이 높은 class 1개를 가져온다.그리고 인덱스만
            answers +=label
            predictions +=prediction
        return predictions,answers

In [ ]:
# Confusion matrix (resnet18)
# kfold의 confusion matrix는 계산 방법이 다르다.
# 모델을 각각 불러와서 test set을 평가한다.

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

cf = np.zeros((2,2))
cf_list = []
average_accuracy = 0
average_fscore = 0

for data_ind in range(1,6):

    check_path = '../checkpoint/checkpoint_w2v_large_finetune_'+str(data_ind)+'.pt'
    model.load_state_dict(torch.load(check_path))

    predictions,answers = test_evaluate(model, test_loader)
    predictions=[ dat.cpu().numpy() for dat in predictions]
    answers=[ dat.cpu().numpy() for dat in answers]

    
    cf = confusion_matrix(answers, predictions)
    cf_list.append(cf)
    
    acc = (cf[0,0]+cf[1,1])/(cf[0,0]+cf[0,1]+cf[1,0]+cf[1,1])
    average_accuracy+=acc
    precision=cf[0,0]/(cf[0,0]+cf[1,0])
    recall=cf[0,0]/(cf[0,0]+cf[0,1])
    #fscore=2*precision*recall/(precision+recall)
    
    #fscroe macro추가
    fscore = f1_score(answers,predictions,average='macro')
    average_fscore+=fscore
    
    print('{}번 모델'.format(data_ind))
    print("Accuracy : {:.4f}% ".format(acc*100))
    print("Precision (pathology 예측한 것중 맞는 것) : {:.4f}".format(precision))
    print("recall (실제 pathology 중  예측이 맞는 것) : {:.4f}".format(recall))
    print("f score : {:.4f} ".format(fscore))
    print(cf)
    print("-----")

print("평균 acc : {:.4f}".format(average_accuracy/5))
print("평균 f1score : {:.4f}".format(average_fscore/5))


